#RepresentationSpace - Discovering Interpretable GAN Controls for Architecture Imagery Synthesis


Using https://github.com/armaank/archlectures/ganspace to find latent directions in a StyleGAN2 model trained on the architecture corpus


## Instructions and Setup

1) Click the play button of the blocks titled "Initialization" and wait for it to finish the initialization.

2) In the Run PCA Analysis section, choose a model, the number of PCA components and the intermediate network layer in the 'Model Specification' cell. Then click the play button to run.  The defaults are ok as is. This block will take a while (~5-10 mins) to run.

3) In the Explore Directions block, generate samples, play with the sliders, and name what you find. In the next block, compare the directions and generate videos.

In [1]:
%%capture
#@title Initialization - Setup
# Clone git
%tensorflow_version 1.x
%rm -rf archlectures
!git clone https://github.com/armaank/archlectures
%cd archlectures/generative/
%ls

In [2]:
#@title Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

KeyboardInterrupt: ignored

In [3]:
#@title Initialization - Download Models
%%capture

%%sh
chmod 755 get_models.sh
./get_models.sh
ls


In [4]:
#@title Initilization - Install Requirements
%%capture

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))
!pip install fbpca boto3
!git submodule update --init --recursive
!python -c "import nltk; nltk.download('wordnet')"
%cd ./ganspace/

from IPython.utils import io
import torch
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
import imageio
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config
from skimage import img_as_ubyte

# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True
# Custom OPs no longer required
#!pip install Ninja
#%cd models/stylegan2/stylegan2-pytorch/op
#!python setup.py install
#!python -c "import torch; import upfirdn2d_op; import fused; print('OK')"
#%cd "/content/ganspace"

## Run PCA Analysis

In [5]:
#@title Model Specification

model = "Adaily_B" #@param ["Adaily_A", "Adaily_B"]
num_components =  80#@param {type:"number"}
layer = 'style'#@param ["style","input","convs","upsamples","noises"]


model_class = model # this is the name of model
model_name = 'StyleGAN2'

!python visualize.py --model $model_name --class $model_class --use_w --layer=style -c $num_components


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.
[08.05 17:18] StyleGAN2, style, ipca
../models/Adaily_B/torch_official/stylegan2_Adaily_1024.pt
Feature shape: torch.Size([1, 512])
Not cached
[08.05 17:18] Computing stylegan2-Adaily_B_style_ipca_c80_n300000_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:289: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:263: FutureWarning: torch.cuda.reset_max_memory_all

## Explore RepresentationSpace

After running the previous cell, your components will be stored in an npz file in `/content/ganspace/cache/components/` - below the npz file is unpacked, and a component/direction is chosen at random. 

Using the UI, you can explore the latent direction and give it a name, which will be appeneded to the named_directions dictionary and saved as `direction_name.npy` for later use.

The variable `seed` controls the starting image

The `Truncation` slider controls the quality of the image sample, .7 is a good starting point

`Scale` is the main slider, it controls the strength/emphasis of the component

`start layer` and `end layer` control the number of layers used in the calculations, using all of them (0, 18) is a good start


In [10]:
#@title Load Model and Component 

config = Config(
  model='StyleGAN2',
  layer=layer,
  output_class=model_class,
  components=num_components,
  use_w=True,
  batch_size=5_000, # style layer quite small
)

inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst)

model = inst.model

named_directions = {} #init named_directions dict to save directions
comps = np.load(path_to_components)
lst = comps.files
latent_dirs = []
latent_stdevs = []

load_activations = False

for item in lst:
    if load_activations:
      if item == 'act_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
            
#load one at random 
num = np.random.randint(20)
if num in named_directions.values():
  print(f'Direction already named: {list(named_directions.keys())[list(named_directions.values()).index(num)]}')

random_dir = latent_dirs[num]
random_dir_stdev = latent_stdevs[num]

print(f'Loaded Component No. {num}')


../models/Adaily_B/torch_official/stylegan2_Adaily_1024.pt
Loaded Component No. 4


In [12]:
#@title Run UI (save component with Enter key)
from ipywidgets import fixed

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def name_direction(sender):
  if not text.value:
    print('Please name the direction before saving')
    return
    
  if num in named_directions.values():
    target_key = list(named_directions.keys())[list(named_directions.values()).index(num)]
    print(f'Direction already named: {target_key}')
    print(f'Overwriting... ')
    del(named_directions[target_key])
  named_directions[text.value] = [num, start_layer.value, end_layer.value]
  save_direction(random_dir, text.value)
  for item in named_directions:
    print(item, named_directions[item])

def save_direction(direction, filename):
  filename += ".npy"
  np.save(filename, direction, allow_pickle=True, fix_imports=True)
  print(f'Latent direction saved as {filename}')

def display_sample_pytorch(seed, truncation, direction, distance, start, end, disp=True, save=None, noise_spec=None, scale=2,):
    # blockPrint()
    with io.capture_output() as captured:
      w = model.sample_latent(1, seed=seed).cpu().numpy()

      model.truncation = truncation
      w = [w]*model.get_max_latents() # one per layer
      for l in range(start, end):
          w[l] = w[l] + direction * distance * scale

      #save image and display
      out = model.sample_np(w)
      final_im = Image.fromarray((out * 255).astype(np.uint8)).resize((500,500),Image.LANCZOS)

    if disp:
      display(final_im)
    if save is not None:
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')

def generate_mov(seed, truncation, direction_vec,  layers, n_frames, out_name = 'out', scale = 2, noise_spec = None, loop=True):
  """Generates a mov moving back and forth along the chosen direction vector"""
  # Example of reading a generated set of images, and storing as MP4.
  %mkdir out
  movieName = f'out/{out_name}.mp4'
  offset = -10
  step = 20 / n_frames
  imgs = []
  for i in log_progress(range(n_frames), name = "Generating frames"):
    print(f'\r{i} / {n_frames}', end='')
    w = model.sample_latent(1, seed=seed).cpu().numpy()

    model.truncation = truncation
    w = [w]*model.get_max_latents() # one per layer
    for l in layers:
      if l <= model.get_max_latents():
          w[l] = w[l] + direction_vec * offset * scale

    #save image and display
    out = model.sample_np(w)
    final_im = Image.fromarray((out * 255).astype(np.uint8))
    imgs.append(out)
    #increase offset
    offset += step
  if loop:
    imgs += imgs[::-1]
  with imageio.get_writer(movieName, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(img_as_ubyte(image))


seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
# scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)
start_layer = widgets.IntSlider(min=0, max=model.get_max_latents(), step=1, value=0, description='start layer: ', continuous_update=False)
end_layer = widgets.IntSlider(min=0, max=model.get_max_latents(), step=1, value=18, description='end layer: ', continuous_update=False)

# Make sure layer range is valid
def update_range_start(*args):
  end_layer.min = start_layer.value
def update_range_end(*args):
  start_layer.max = end_layer.value
start_layer.observe(update_range_start, 'value')
end_layer.observe(update_range_end, 'value')

text = widgets.Text(description="Name component here", style=style, width=200)

bot_box = widgets.HBox([seed, truncation, distance, start_layer, end_layer, text])
ui = widgets.VBox([bot_box])

out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance,'start': start_layer, 'end': end_layer})

display(ui, out)
text.on_submit(name_direction)

Output()

Latent direction saved as a.npy
c [4, 4, 15]
a [4, 4, 14]
Latent direction saved as daf.npy
c [4, 4, 15]
a [4, 4, 14]
daf [4, 8, 15]


In [13]:
#@title Select from named directions

from IPython.display import display, clear_output

vardict = list(named_directions.keys())
select_variable = widgets.Dropdown(
    options=vardict,
    value=vardict[0],
    description='Select variable:',
    disabled=False,
    button_style=''
)

def set_direction(b):
    clear_output()
    random_dir = latent_dirs[named_directions[select_variable.value][0]]
    start_layer = named_directions[select_variable.value][1]
    end_layer = named_directions[select_variable.value][2]
    print(start_layer, end_layer)
    out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})
    display(select_variable)
    display(ui, out)

random_dir = latent_dirs[named_directions[select_variable.value][0]]
start_layer = named_directions[select_variable.value][1]
end_layer = named_directions[select_variable.value][2]
seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)

bot_box = widgets.HBox([seed, truncation, distance])
ui = widgets.VBox([bot_box])
out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})

display(select_variable)
display(ui, out)

select_variable.observe(set_direction, names='value')



4 14


Dropdown(description='Select variable:', index=1, options=('c', 'a', 'daf'), value='a')

Output()

In [15]:
#@title Generate Video from Representation (Optional)
direction_name = "a" #@param {type:"string"}
num_frames =  5 #@param {type:"number"}
truncation = 0.8 #@param {type:"number"}

num_samples = num_frames
assert direction_name in named_directions, \
  f'"{direction_name}" not found, please save it first using the cell above.'

loc = named_directions[direction_name][0]
for i in range(num_samples):
  s = np.random.randint(0, 10000)
  generate_mov(seed = s, truncation = 0.8, direction_vec = latent_dirs[loc], scale = 2, layers=range(named_directions[direction_name][1], named_directions[direction_name][2]), n_frames = 20, out_name = f'{model_class}_{direction_name}_{i}', loop=True)

print('Video saved to ./ganspace/out/')

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


...

mkdir: cannot create directory ‘out’: File exists


19 / 20

mkdir: cannot create directory ‘out’: File exists


19 / 20

Video saved to ./ganspace/out/
